In [33]:
# Electricity & Gas Consumption Heat Map by California Counties
# Energy Usage Data Resource from California Energy Commission (https://www.energy.ca.gov/)
# County Border Resource from Opendatasoft (https://hifld-dhs-gii.opendata.arcgis.com/datasets/53270fe7036f428fbfb11c4511994a6f_0)

# Importing Block
import pandas as pd
import folium

In [35]:
# All Usage Expressed in Millions of kWh (GWh)
# All electricity usages are total of residential and non-residential

###### Note
"""
1. Keep thinking about what needs to be done
2. Make county border lines including a tooltip that contains Total of and an average of energy usages + energy heatmap indicator + county name

- Extract values from 'coordinates' (json? dict?, type = np.ndarray with one dimension) and add to the map <--- (in-process)

"""
######


### Data Path Block ###
usage_df = pd.read_csv('/Users/jk/Desktop/A/Study/github/Data-Analysis-Project/Energy Consumption Heatmap by California Counties/ElectricityByCounty_1990_2019.csv')
county_df = pd.read_csv('/Users/jk/Desktop/A/Study/github/Data-Analysis-Project/Energy Consumption Heatmap by California Counties/us-county-boundaries.csv', sep = ';')



### Data Formatting Block ###
usage_df.drop(columns = ['Sector'], 
              inplace = True) # To drop 'Sector' (value: Total) column

years = list(map(str, range(1990, 2020))) # Convert columns of each year to str value for line ???
usage_df[years] = usage_df[years].astype(int)

ca_county = county_df[county_df['STUSAB'] == 'CA']
ca_county = ca_county[['NAME', 
                       'Geo Point', 
                       'Geo Shape']].sort_values(by = 'NAME')

ca_county.set_index([pd.Index(list(map(int, range(1, 59))))], inplace = True) # Add index 1 - 58

ca_county['Geo Point'].replace(to_replace = ca_county.loc[38]['Geo Point'], 
                                          value = '37.773972,-122.431297', # Fixed San Francisco coordinates that was not centered on the map
                                          inplace = True) 



### Folium Setting Block ###
ca_map = folium.Map(location = [36.7783, -119.4179],
                    width = 640, 
                    height = 700, 
                    zoom_start = 6, 
                    min_zoom = 6) # California State latitude, longitude

df_lat = []
df_lon = []
county_name = [n for n in ca_county['NAME']]

for x in ca_county['Geo Point']: # Iterate latitude and longitude to location parameter of marker
    for i in x.split(','):
        i = float(i)
        if i > 0:
            df_lat.append(i)
        else:
            df_lon.append(i)

zipped_county_loc_name = zip(df_lat, df_lon, county_name)

for lat, lon, name in zipped_county_loc_name:
    county_marker = folium.Marker(location = [lat, lon], popup = name, tooltip = name).add_to(ca_map)

ca_map

# county_border = folium.Popup(locations = [aa]).add_to(ca_map)
